In [2]:
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout,  Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2, l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests

In [3]:
stock_data = [[]]
df = [[]]
ticker= 'AMZN'
date = datetime.now().strftime("%Y%m%dT0000")
date

'20250326T0000'

In [4]:
# Get the current timestamp in the required format
ticker= 'AMZN'
date = (datetime.now() - timedelta(hours=24)).strftime("%Y%m%dT0000")
print(date)

# Construct the API URL  (NP16WASG3Q4KMP82) (WM25MK3BXKH6YHTQ) (J41PMO618QEF01VL)
api_url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&time_from={date}&limit=100000&apikey=NP16WASG3Q4KMP82'

# Make the API request
response = requests.get(api_url)
news_data = response.json()
news_data

20250325T0000


{'items': '24',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': "Amazon's AI Cloud Strategy To Stay Strong Amid 'Near-Term Turbulence', Says Analyst: 'AWS Remains Well-Positioned For Long-Term AI Trends' - Alphabet  ( NASDAQ:GOOG ) , Amazon.com  ( NASDAQ:AMZN ) ",
   'url': 'https://www.benzinga.com/25/03/44488542/amazons-ai-cloud-strategy-to-stay-strong-amid-near-term-turbulence-says-analyst-aws-remains-well-positioned-for-l',
   'time_published': '20250326T084726',
   'authors': ['Namrata Sen'],
   'summary': 'Amazon Inc. AMZN has been lauded by Monness Crespi Hardt analyst for its positioning in the AI cloud sector, despite the weakness in other technology companies. What Happened: The GTC AI conference hosted by Nvidia NVDA, dubbed the "Super B

In [5]:
# Extract news articles
articles = news_data.get('feed', [])
# Check if data is available before creating DataFrame
if not articles:
    print("No articles found.")
else:
    # Normalize JSON structure to flatten nested fields
    df = pd.json_normalize(articles, 'ticker_sentiment', ['title'], errors='ignore')

    # Filter for AAPL only and select relevant columns
    df = df[df['ticker'] == 'AAPL'][['title', 'ticker_sentiment_score']]

    # Display the processed DataFrame
    print(df.head())


                                                title ticker_sentiment_score
8   The "Magnificent Seven" Stocks Are Selling Off...               0.047753
19  Big Tech Companies Employ Majority Of H-1B Vis...               0.144533
27  Spotify Stock Soars 130% in a Year: Is it Stil...               0.145919
38  Live Nation Entertainment Rises 22% in a Year:...               0.169758
45  Netflix's Content Strategy Signals Strong 2025...               0.207115


In [11]:
# Fetch stock data using yfinance
def fetch_stock_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data[['Close', 'Open', 'High', 'Low', 'Volume']]

# Load the dataset
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
stock_data = fetch_stock_data(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [12]:
stock_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2024-02-01 to 2025-01-31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   251 non-null    float64
 1   Open    251 non-null    float64
 2   High    251 non-null    float64
 3   Low     251 non-null    float64
 4   Volume  251 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 11.8 KB


In [13]:
stock_data.isnull().sum()

Close     0
Open      0
High      0
Low       0
Volume    0
dtype: int64

In [1]:
stock_data['SMA_50'] = ta.sma(stock_data['Close'], length=50).fillna(0)
stock_data['SMA_200'] = ta.sma(stock_data['Close'], length=200).fillna(0)
stock_data['EMA_12'] = ta.ema(stock_data['Close'], length=12).fillna(0)
stock_data['EMA_26'] = ta.ema(stock_data['Close'], length=26).fillna(0)
stock_data['RSI'] = ta.rsi(stock_data['Close'], length=14).fillna(0)
macd = ta.macd(stock_data['Close'], fast=12, slow=26, signal=9).fillna(0)
stock_data['MACD'] = macd['MACD_12_26_9'].fillna(0)
stock_data['MACD_signal'] = macd['MACDs_12_26_9'].fillna(0)
bollinger = ta.bbands(stock_data['Close'], length=20)
stock_data['UpperBand'] = bollinger['BBU_20_2.0'].fillna(0)
stock_data['MiddleBand'] = bollinger['BBM_20_2.0'].fillna(0)
stock_data['LowerBand'] = bollinger['BBL_20_2.0'].fillna(0)



NameError: name 'ta' is not defined

In [23]:
stock_data.isnull().sum()

Close          0
Open           0
High           0
Low            0
Volume         0
SMA_50         0
SMA_200        0
EMA_12         0
EMA_26         0
RSI            0
MACD           0
MACD_signal    0
UpperBand      0
MiddleBand     0
LowerBand      0
dtype: int64

In [24]:
print(stock_data.head())

                 Close        Open        High         Low     Volume  SMA_50  \
Date                                                                            
2024-02-01  159.279999  155.869995  159.759995  155.619995   76542400     0.0   
2024-02-02  171.809998  169.190002  172.500000  167.330002  117154900     0.0   
2024-02-05  170.309998  170.199997  170.550003  167.699997   55081300     0.0   
2024-02-06  169.149994  169.389999  170.710007  167.649994   42505500     0.0   
2024-02-07  170.529999  169.479996  170.880005  168.940002   47174100     0.0   

            SMA_200  EMA_12  EMA_26  RSI  MACD  MACD_signal  UpperBand  \
Date                                                                     
2024-02-01      0.0     0.0     0.0  0.0   0.0          0.0        0.0   
2024-02-02      0.0     0.0     0.0  0.0   0.0          0.0        0.0   
2024-02-05      0.0     0.0     0.0  0.0   0.0          0.0        0.0   
2024-02-06      0.0     0.0     0.0  0.0   0.0          0.0   

In [57]:
# Ensure stock_data and df have the same length before assignment
print(stock_data.count().sum())
print(df.count().sum())
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df)
min_length = min(len(stock_data), len(df))
df['ticker_sentiment_score'] = pd.to_numeric(df['ticker_sentiment_score'], errors='coerce').fillna(0)
# Assign only matching length values
if len(stock_data) == min_length:        # Trim stock_data if needed
    df = df.iloc[:min_length]  
else:                                    # Trim df if needed
    stock_data = stock_data.iloc[:min_length]
stock_data['Sentiment'] = df['ticker_sentiment_score'].values
print(min_length)
print("----------------------------------------------------------------------------")
print(stock_data.head())

38
4
                                                title  ticker_sentiment_score
4   EXCLUSIVE: This 'Magnificent 7' Stock Will Fee...                0.000000
33  Over 25% of Warren Buffett's $300 Billion Port...                0.359071
2
----------------------------------------------------------------------------
                 Close        Open        High         Low     Volume  SMA_50  \
Date                                                                            
2024-02-01  159.279999  155.869995  159.759995  155.619995   76542400     0.0   
2024-02-02  171.809998  169.190002  172.500000  167.330002  117154900     0.0   

            SMA_200  EMA_12  EMA_26  RSI  MACD  MACD_signal  UpperBand  \
Date                                                                     
2024-02-01      0.0     0.0     0.0  0.0   0.0          0.0        0.0   
2024-02-02      0.0     0.0     0.0  0.0   0.0          0.0        0.0   

            MiddleBand  LowerBand  Sentiment  Close_Lag1  Cl

In [29]:
# Add lag features
stock_data['Close_Lag1'] = stock_data['Close'].shift(1).fillna(0)
stock_data['Close_Lag2'] = stock_data['Close'].shift(2).fillna(0)
stock_data['Daily_Change'] = (stock_data['Close'] - stock_data['Open']) / stock_data['Open']
print(stock_data.head())

                 Close        Open        High         Low     Volume  SMA_50  \
Date                                                                            
2024-02-01  159.279999  155.869995  159.759995  155.619995   76542400     0.0   
2024-02-02  171.809998  169.190002  172.500000  167.330002  117154900     0.0   

            SMA_200  EMA_12  EMA_26  RSI  MACD  MACD_signal  UpperBand  \
Date                                                                     
2024-02-01      0.0     0.0     0.0  0.0   0.0          0.0        0.0   
2024-02-02      0.0     0.0     0.0  0.0   0.0          0.0        0.0   

            MiddleBand  LowerBand  Sentiment  Close_Lag1  Close_Lag2  \
Date                                                                   
2024-02-01         0.0        0.0   0.000000    0.000000         0.0   
2024-02-02         0.0        0.0   0.359071  159.279999         0.0   

            Daily_Change  
Date                      
2024-02-01      0.021877  
2024-02-

In [30]:
X = stock_data[['Open', 'High', 'Low', 'Volume', 'SMA_50', 'SMA_200', 'EMA_12', 'EMA_26', 'RSI', 
                'MACD', 'MACD_signal', 'UpperBand', 'MiddleBand', 'LowerBand', 
                'Close_Lag1', 'Close_Lag2', 'Daily_Change', 'Sentiment']]
y = stock_data['Close']

In [43]:
# Scale the data
scaler_X = StandardScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Create sequences
def create_sequences(X, y, time_steps):
    X_seq, y_seq = [], []
    print(f"Total samples in X: {len(X)}, Time steps: {time_steps}")
    for i in range(len(X) - time_steps):
        X_seq.append(X[i:i + time_steps])
        y_seq.append(y[i + time_steps])
    return np.array(X_seq), np.array(y_seq)

time_steps = 1
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)
print(X.shape)
print(y.shape)
print(X_seq.shape, y_seq.shape)



Total samples in X: 2, Time steps: 1
(2, 18)
(2,)
(1, 1, 18) (1, 1)


In [44]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.25, random_state=42, shuffle=False)

ValueError: With n_samples=1, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [33]:

# Build the LSTM model
model = Sequential([
    Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(time_steps, X.shape[1]))),
    Dropout(0.3),
    Bidirectional(LSTM(units=32)),
    Dropout(0.3),
    Dense(units=1)
])

# Learning rate scheduler
lr_schedule = ExponentialDecay(initial_learning_rate=0.001, decay_steps=1000, decay_rate=0.95)
optimizer = Adam(learning_rate=lr_schedule)

# Compile the model
model.compile(optimizer=optimizer, loss='huber_loss')

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=50, batch_size=32, verbose=1, 
                    callbacks=[early_stopping, model_checkpoint])

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_inverse = scaler_y.inverse_transform(y_pred)
y_test_inverse = scaler_y.inverse_transform(y_test)

try:
    next_day_features = X_test[-1].reshape(1, time_steps, X.shape[1])  # Ensure proper shape
    next_day_prediction = model.predict(next_day_features)
    next_day_prediction = scaler_y.inverse_transform(next_day_prediction)  # Rescale to original range

    # Display the prediction
    print("Predicted next day's stock price:", next_day_prediction[0][0])
except Exception as e:
    print(f"Error during prediction: {e}")

NameError: name 'X_train' is not defined